# PR 4.3
Van Pjotr en Sennen

Hierin gaan wij een paar queries aanvragen aan de database die wij hebben gemaakt gebasseerd op de ETL diagram van de Great_Outdoors.

Hieronder zullen we beginnen met de setup van de libraries en het verbinden met de database. We zullen de pyodbc library gebruiken om de verbinding tussen SSMS en Python. Verder gebruiken we ook pandas om data makkelijk te lezen.

In [1]:
import pandas as pd
import pyodbc
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")
load_dotenv()

True

In [2]:
DB = {'servername': os.getenv('NAME'),
      'database': os.getenv('DATABASE'),
      'username': os.getenv('USER'),
      'password': os.getenv('PASSWORD')}

conn_str = f"DRIVER=SQL Server;SERVER={DB['servername']};DATABASE={DB['database']};UID={DB['username']};PWD={DB['password']};Trusted_Connection=yes;"

conn = pyodbc.connect(conn_str)
export_cursor = conn.cursor()
# Hoe checken we of de connectie werkt?
print(export_cursor.execute("SELECT @@version;"))

Om ervoor te zorgen dat we weten dat we data kunnen aanvragen vanaf de server zullen we hier een paar queries executeren. Zodat we kunnen bevestigen dat we alles goed hebben geconfigureerd. Eerst pakken we alle tabelen van uit de database.

In [3]:
export_cursor.execute("SELECT t.name FROM sys.tables t")
tables = export_cursor.fetchall()

# Voor elke tabel in de database print de naam van de tabel
if(tables == []):
    print("No tables found, the database is empty.")
else:
    for table in tables:
        table = table[0]
        print(table[0])

No tables found, the database is empty.


Daarna wat we willen zijn dus de functies maken die we later in het project gaan gebruiken om de data in te laden in ons project. Zodat we de data uit de bron kunnen transformeren en overzetten naar onze SQL Server database. We maken hiervoor een extract functie maken en een laad functie.

## Extract
Eerst gaan we de data uit de access database halen en ervoor zorgen dat we ze later goed kunnen transformeren in de database.

In [5]:
# Verbind met sqlite go_sales staff
conn_access = create_engine('sqlite:///go_sales.sqlite')
product = pd.read_sql("SELECT * FROM product", conn_access)
product_type = pd.read_sql("SELECT * FROM product_type", conn_access)
product_line = pd.read_sql("SELECT * FROM product_line", conn_access)
ProductMerge1 = pd.merge(product, product_type, how="left", left_on="PRODUCT_TYPE_CODE", right_on="PRODUCT_TYPE_CODE")
ProductExtract = pd.merge(ProductMerge1, product_line, how="left", left_on="PRODUCT_LINE_CODE", right_on="PRODUCT_LINE_CODE")
ProductExtract

,PRODUCT_NUMBER,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION,TRIAL888_x,PRODUCT_LINE_CODE,PRODUCT_TYPE_EN,TRIAL888_y,PRODUCT_LINE_EN,TRIAL888
0,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,1,Cooking Gear,T,Camping Equipment,T
1,10,15-2-2011,1,10,.4,P10CE1CG1.jpg,EN,TrailChef Utensils,"Spoon, fork and knife set made of a light yet ...",T,1,Cooking Gear,T,Camping Equipment,T
2,100,15-2-2011,17,3,.5,P100OP4FA17.jpg,EN,Insect Bite Relief,The Insect Bite Relief helps the itching and s...,T,4,First Aid,T,Outdoor Protection,T
3,101,15-12-2019,18,305.54,.43,P101GE5IR18.jpg,EN,Hailstorm Steel Irons,Iron is 17-4 stainless steel. Shafts are grap...,T,5,Irons,T,Golf Equipment,T
4,102,10-12-2019,18,380.95,.51,P102GE5IR18.jpg,EN,Hailstorm Titanium Irons,Made entirely of pure titanium. The ultimate i...,T,5,Irons,T,Golf Equipment,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,95,15-2-2011,16,3,.5,P91OP4SS16.jpg,EN,Sun Shield,"PABA free sunscreen, SPF 30, poison oak and iv...",T,4,Sunscreen,T,Outdoor Protection,T
111,96,15-2-2011,17,16.43,.28,P96OP4FA17.jpg,EN,Compact Relief Kit,A personal first aid kit is recommended for ev...,T,4,First Aid,T,Outdoor Protection,T
112,97,5-3-2013,17,25,.28,P96OP4FA17.jpg,EN,Deluxe Family Relief Kit,A complete medical kit suitable for families w...,T,4,First Aid,T,Outdoor Protection,T
113,98,15-2-2011,17,3,.5,P98OP4FA17.jpg,EN,Calamine Relief,Use the Calamine Relief for allergic skin reac...,T,4,First Aid,T,Outdoor Protection,T


## Transform
Nadat we de data eruit hebben gehaald, gaan de data transformeren, zodat we ze in de database kunnen stoppen.

## Loading

Hieronder gaan we de data inladen. Hieronder zal ik de code beschikbaar zijn.